# 00 Init

## Mount

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setting to use py files

In [13]:
import os

In [14]:
os.chdir('/content/drive/MyDrive/Minesweeper [RL]')

In [15]:
# check that os is in right directory
os.getcwd()

'/content/drive/MyDrive/Minesweeper [RL]'

In [16]:
! pip install codes

## Import py files

In [17]:
# baseline : Env, Agent
from codes.Agent.DQNAgentWithoutRules import *
from codes.Trainer.trainerWithoutRules import *
from codes.Environment.env5reward import *

## Import Libraries

# 01 Info

## level dictionary

In [18]:
level = {'easy' : {'map_size':(9,9), 'n_mines' : 10},
         'medium' : {'map_size':(16,16), 'n_mines':40},
         'expert' : {'map_size':(16,30), 'n_mines':99}}

## HYPER PARAMETERS

In [19]:
# Environment settings
MEM_SIZE = 50000
MEM_SIZE_MIN = 1000

# Learning settings
BATCH_SIZE = 64
LEARNING_RATE = 0.01
LEARN_DECAY = 0.999975
LEARN_MIN = 0.001
DISCOUNT = 0.1

# Exploration settings
EPSILON = 0.95
EPSILON_DECAY = 0.99975
EPSILON_MIN = 0.01

# DQN settings
CONV_UNITS = 64
UPDATE_TARGET_EVERY = 5

# 02 Train, Valid

In [20]:
env = MinesweeperEnv(map_size=level['easy']['map_size'],
                     n_mines=level['easy']['n_mines'],
                     rewards={'win':1, 'lose':-1, 'progress':0.3, 'guess':-0.3, 'no_progress' : -0.5})
agent = Limited18DQNAgent(env=env, conv_units=CONV_UNITS)

## TRAIN_PARAMETERS

In [10]:
EPISODES = 50000
PRINT_INTERVAL = 100
TRAIN_RENDER = False

TRAIN_INTERVALS = ['every timestep', 'every episodes']
TRAIN_INTERVAL = TRAIN_INTERVALS[0]

VALID_SAMPLE = 1000
VALID_INTERVAL = 10

In [ ]:
train = Trainer(env, agent, 'basicModelWithBiasWithoutRule')

Episode: [50000/200], Median progress: 8.00, Median reward: -2.65, Win rate : 0.00, Epsilon: 0.83
Episode: [50000/300], Median progress: 7.00, Median reward: -2.65, Win rate : 0.00, Epsilon: 0.66
Episode: [50000/400], Median progress: 7.00, Median reward: -2.25, Win rate : 0.00, Epsilon: 0.52
Episode: [50000/500], Median progress: 6.00, Median reward: -2.05, Win rate : 0.00, Epsilon: 0.38
Episode: [50000/600], Median progress: 6.00, Median reward: -2.05, Win rate : 0.00, Epsilon: 0.28
Episode: [50000/700], Median progress: 6.00, Median reward: -1.30, Win rate : 0.00, Epsilon: 0.21
Episode: [50000/800], Median progress: 5.00, Median reward: -1.30, Win rate : 0.00, Epsilon: 0.16
Episode: [50000/900], Median progress: 5.00, Median reward: -1.30, Win rate : 0.00, Epsilon: 0.12
Episode: [50000/1000], Median progress: 6.00, Median reward: -1.30, Win rate : 0.00, Epsilon: 0.08
Episode: [50000/1100], Median progress: 4.50, Median reward: -1.30, Win rate : 0.00, Epsilon: 0.04
Episode: [50000/12

In [ ]:
train.train()

# 03 Test